# Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

In [ ]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 0) Setup

In [ ]:
# Define our project information
EXPERIMENT = "rush qm geo opt notebook"
MOLECULE_NAME = "cyclobutane"
TAGS = ["qdx", EXPERIMENT, MOLECULE_NAME]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
os.makedirs(WORK_DIR, exist_ok=True)

## 0.1) Initialize our rush client and fetch available module paths

In [ ]:
# By using the `build_provider_with_functions` method, we will also build
# helper functions calling each module
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(batch_tags=TAGS, workspace=WORK_DIR, restore_by_default=True)

## 0.2) Get qm_geo_opt rush module

In [ ]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducible
# This will be done automatically if you use the `build_provider_with_functions`
# method
modules = await client.get_latest_module_paths()

In [ ]:
for name, path in sorted(modules.items()):
    print(f"{name}: {path}")

auto3d: github:talo/tengu-auto3d/f5f26308f74bde570c870ff0b5bb1be42ac9c712#auto3d_tengu
cairo: github:talo/cairo/665b83a4ed8ff5e811fcbd9e55ddf8f21e820aeb#cairo_tengu
concat: github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#concat
conquest: github:talo/CONQUEST-tengu/c668c2c9f364835237f9052ab4721c29a2e0d7ac#conquest_tengu
convert: github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#convert
delay: github:talo/tengu-module-example/2ca14294027c7c20b7f2c8086c2c9cf5f953e1b8#delay
dubai: github:talo/Dubai/cc28be9a53b1aa958cd2aab753223daa698296ce#dubai_tengu
echo: github:talo/tengu-module-example/b334a851530033b79762c3341bf584f8939feee1#tengu_echo
fragment: github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#fragment
fragment_aa: github:talo/tengu-prelude/efc6d8b3a8cc342cd9866d037abb77dac40a4d56#fragment_aa
gmx: github:talo/gmx_tengu_support/92b92afa4ad344e384d263d829ee1d58da1dc17c#gmx_tengu
gmx_em: github:talo/gmx_tengu_support/1c8beb04620306f991

In [ ]:
help(client.qm_geo_opt)

Help on function qm_geo_opt in module rush.provider:

async qm_geo_opt(*args: *tuple[RushObject[Conformer], Record], target: Optional[Target] = None, resources: Optional[Resources] = {'storage': 1034, 'storage_units': 'MB', 'gpus': 4}, tags: list[str] | None = None, restore: bool | None = None) -> tuple[RushObject[list[Conformer]], RushObject[Record]]
    geometry optimise molecule with quantum accuracy
    
    Module version:  
    `github:talo/qm_geo_opt/3bc72f5b9204ad67e317d6921b2eec461c1fa0a9#qm_geo_opt_tengu`
    
    QDX Type Description:
    
        input: @Conformer;
        parameters: {
            basis_set: string,
            conv_threshold: f64,
            max_iterations: u32,
            use_internal_coords: bool
        }
        ->
        conformers: @[Conformer];
        geo_opt_results: @{
            converged: bool,
            final_energy: f64,
            final_gradient_error: f64,
            iterations_to_converge: u32
        }
    
    :param input: the 

# 1) Run Geometry Optimisation

## 1.0) Create input geometry
We will be creating a QDXF Conformer from an unoptimised cyclobutane XYZ

In [ ]:
def xyz_to_qdxf_topology(xyz_data: str):
    symbols = []
    geometry = []
    for line in xyz_data.splitlines()[2:]:
        symbol, x, y, z = line.split()
        symbols.append(symbol)
        geometry.extend([float(x), float(y), float(z)])
    return {"topology": {"symbols": symbols, "geometry": geometry}}


def qdxf_topology_to_xyz(qdxf_in: dict, name: str):
    symbols = qdxf_in["topology"]["symbols"]
    geometry = qdxf_in["topology"]["geometry"]
    xyz_data = f"{len(symbols)}\n{name}\n"
    for i in range(len(symbols)):
        symbol = symbols[i]
        x, y, z = geometry[3 * i : 3 * i + 3]
        xyz_data += f"  {symbol}"
        for coord in [x, y, z]:
            xyz_data += "     "
            if coord >= 0:
                xyz_data += " "
            xyz_data += f"{(coord):.4f}"
        xyz_data += "\n"
    return xyz_data

In [ ]:
cyclobutane_xyz = """12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
"""

In [ ]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [ ]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
    json.dump(cyclobutane_qdxf, f)

## 1.1) Create geometry optimisation parameters

In [ ]:
params = {
    "use_internal_coords": True,
    "max_iterations": 100,
    "conv_threshold": 1e-4,
    "basis_set": "cc-pVDZ",
}

## 1.1) Call geometry optimisation over input geometry

In [ ]:
QM_GEO_OPT_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
(geo_opt_out, results) = await client.qm_geo_opt(
    cyclobutane_qdxf_input_file, params, resources=QM_GEO_OPT_RESOURCES
)
print(await geo_opt_out.get())

2024-02-10 20:56:07,269 - rush - INFO - Trying to restore job with tags: ['qdx', 'rush qm geo opt notebook', 'cyclobutane'] and path: github:talo/qm_geo_opt/3bc72f5b9204ad67e317d6921b2eec461c1fa0a9#qm_geo_opt_tengu
2024-02-10 20:56:07,518 - rush - INFO - Restoring job from previous run with id 5855e05b-5444-4756-bd1a-c186248f294b
[{'topology': {'geometry': [-0.5669037, 0.70684737, -0.17732358, -0.079094425, 0.08392154, 1.1556711, 0.4376942, -1.1220864, 0.3301761, 0.47652462, -0.18179193, -0.9015664, -0.512153, 1.7981342, -0.30335212, -1.5895649, 0.36984435, -0.41000292, -0.8121022, -0.096688, 1.9555683, 0.76496166, 0.6610957, 1.5655093, -0.35436994, -1.8788438, 0.21416105, 1.3661636, -1.6205282, 0.6452785, 0.22627495, -0.5936566, -1.8902533, 1.4475691, 0.33475187, -0.962866], 'symbols': ['C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']}}]


## 1.2) Check calculation converged

In [ ]:
results_out = await results.get()
assert results_out["converged"]

# 2) Visualise pre-optimised and optimised geometries

In [ ]:
import py3Dmol

In [ ]:
pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({"stick": {}})
pre_opt_view.zoomTo()
pre_opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
topologies_out = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(
    topologies_out[-1], "optimised cyclobutane"
)

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({"stick": {}})
opt_view.zoomTo()
opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol